In [3]:
import pandas as pd

In [4]:
def reg_pay_id(df):
    """
    Returns dict showing regular creditors along with average gaps between payments and average amounts.
    Can be used to generate suggestions for auto-saving pots of money for specific purposes.
    """
    df['dates'] = pd.to_datetime(df['dates'])
        
    df = df.reset_index()    
    main_dict = {}
    for party in list(df['other_account_name'].value_counts().index):
        temp_df = df[df['other_account_name'] == party].copy()
        temp_df['time_since_last_trans_party'] = temp_df['dates'] - temp_df['dates'].shift()        
        temp_df = temp_df.reset_index()
        pos_dict = pd.Series(temp_df['time_since_last_trans_party'].values, index = temp_df['index'])
        main_dict.update(pos_dict)
    df['time_since_last_trans_party'] = df['index'].map(main_dict)    
    
    # Select only Regular payments - can be changed to use cluster tag
    df_reg = df.loc[df['Type'] == 'Regular']
    parties = df_reg['other_account_name'].unique()
    reg_dict = {}
    for party in parties:    
        df_p = df_reg.loc[df_reg['other_account_name'] == party]
        gaps = list(df_p['time_since_last_trans_party'].value_counts().index)                
        if len(gaps) > 0:
            gaps = [x.days for x in gaps if len(gaps) > 0]           
            avg_gap = int(sum(gaps)/len(gaps))        
        else:
            avg_gap = None
        
        amounts = list(df_p['amount'].value_counts().index)  
        amounts = [float(x) for x in amounts]
        avg_amount = sum(amounts)/len(amounts)        
        
        reg_dict.update({party:[avg_gap,round(avg_amount,2)]})
        
    return reg_dict

In [40]:
df = pd.read_csv('gs://shakingshamrocks_eu/test_data_3_sec.csv')

In [41]:
# Drop first column (index)
df = df.drop(df.columns[0],axis = 1)

In [44]:
df.dtypes

dates                  object
account_name           object
account_IBAN           object
other_account_name     object
Type                   object
amount                float64
balance               float64
dtype: object

In [47]:
df['minus'] = df.loc[df['amount'].apply(lambda x: True if x < 0 else False)]

ValueError: Wrong number of items passed 7, placement implies 1

In [37]:
df.head()

,dates,account_name,account_IBAN,other_account_name,Type,amount,balance
0,2018-02-16 02:17:13,Katherine Valencia,DOJO46309747392475,Wages MyOrg,Regular,2300.000000,2300.000000
1,2018-02-20 14:29:42,Patrick Hamilton,CJVO60129798466874,Wages MyOrg,Regular,2600.000000,4900.000000
2,2018-02-25 21:12:59,Katherine Valencia,DOJO46309747392475,Food Reg,Irregular,-220.577729,4679.422271
3,2018-02-28 01:18:39,Katherine Valencia,DOJO46309747392475,Subscription,Regular,-55.000000,4624.422271
4,2018-02-28 03:02:54,Katherine Valencia,DOJO46309747392475,Phone,Regular,-18.000000,4606.422271


In [38]:
df_test = df.loc[df['account_name'] == 'Katherine Valencia']

In [39]:
df_test.head()

,dates,account_name,account_IBAN,other_account_name,Type,amount,balance
0,2018-02-16 02:17:13,Katherine Valencia,DOJO46309747392475,Wages MyOrg,Regular,2300.000000,2300.000000
2,2018-02-25 21:12:59,Katherine Valencia,DOJO46309747392475,Food Reg,Irregular,-220.577729,4679.422271
3,2018-02-28 01:18:39,Katherine Valencia,DOJO46309747392475,Subscription,Regular,-55.000000,4624.422271
4,2018-02-28 03:02:54,Katherine Valencia,DOJO46309747392475,Phone,Regular,-18.000000,4606.422271
7,2018-02-28 07:15:31,Katherine Valencia,DOJO46309747392475,Car,Regular,-330.000000,2572.323012


In [ ]:
json_data = df_test.to_json()
r = requests.post('https://demo-app-lquvhriy2a-ew.a.run.app/service/classify_v2/',  json= {"data":json_data})
df_results = pd.read_json(r.json())
df_results.head()

In [17]:
import json

In [36]:
reg_pay_id(df)

{'Wages MyOrg': [14, 2450.0],
 'Subscription': [14, -52.5],
 'Phone': [14, -19.0],
 'Rent - apt 12': [14, -1350.0],
 'Utilities': [29, -238.92],
 'Car': [14, -290.0],
 'Medical Session': [29, -200.0],
 'Medical Subscription': [14, -115.0]}

In [22]:
json.dumps(reg_pay_id(df))

'{"Wages MyOrg": [14, 2450.0], "Medical Subscription": [14, -115.0], "Medical Session": [29, -200.0], "Car": [14, -290.0], "Utilities": [29, -242.36], "Rent - apt 12": [14, -1350.0], "Phone": [14, -19.0], "Subscription": [14, -52.5]}'

In [23]:
import requests

In [35]:
json_data = df.to_json()
r = requests.post('https://demo-app-lquvhriy2a-ew.a.run.app/service/intelligent/saving_v2/',  json= {"data":json_data})
r.json()

{'Wages MyOrg': [14, 2450.0],
 'Subscription': [14, -52.5],
 'Phone': [14, -19.0],
 'Rent - apt 12': [14, -1350.0],
 'Utilities': [29, -238.92],
 'Car': [14, -290.0],
 'Medical Session': [29, -200.0],
 'Medical Subscription': [14, -115.0]}

In [29]:
r

<Response [200]>